In [6]:
!pip install nltk

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 1.0 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 838.9 kB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 838.9 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 853.0 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 853.0 kB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 780.2 kB/s eta 0:00:01
   ---------------------------------------- 1.5/1.

In [129]:
!pip install scikit-learn


  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl (10.9 MB)
Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [155]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
import sklearn

In [156]:
txt = "Check out this link: https://example.com! It's amazing, right? 123"

In [168]:
import pickle
with open('../static/model/model.pickle','rb')as f:
    model=pickle.load(f)

C:\Users\Pasindu\Desktop\Sentiment-Analysis\env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [169]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens= vocab[0].tolist()

In [157]:
with open ('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()
string.punctuation

punctuation_pattern = r'[^\w\s]' 
number_pattern = r'\d+'

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [158]:
stop_words = set(stopwords.words('english'))
def preprocessing(text):
    data = pd.DataFrame([text],columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(re.sub(r'^https?:\/\/.*[\r\n]*', '',x,flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].str.replace(punctuation_pattern, '', regex=True)
    data['tweet'] = data['tweet'].str.replace(number_pattern, '', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(stemmer.stem(x) for x in x.split()))
    return data['tweet']

In [159]:
preprocessed_txt = preprocessing(txt)
preprocessed_txt

0    check link amaz right
Name: tweet, dtype: object

In [160]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens= vocab[0].tolist()

In [162]:
def vectorizer(ds,vocabulary):
    vectorized_lst=[]
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] =1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.float32)
    if len(ds)==1:
        return vectorized_lst_new[0]
    return vectorized_lst_new

In [170]:
vectorized_txt=vectorizer(preprocessed_txt,tokens)


In [171]:
def get_prediction(vectorized_txt):
    prediction = model.predict([vectorized_txt])
    if prediction == 1:
        return 'negative'
    else:
        return 'Positive'

get_prediction(vectorized_txt)

'Positive'